## Import libraries

In [0]:
import os
import zipfile
from azure.storage.blob import BlobServiceClient
from pyspark.dbutils import DBUtils
from azure.storage.blob import BlobServiceClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import col, expr

## Getting Data from Datalake and do exploration

In [0]:
# Set Azure Storage credentials
AZURE_CONN_STRING = 'DefaultEndpointsProtocol=https;AccountName=noviprojectst;AccountKey=AQP6/OKhC0l5El39RPZ72SUQr65WvmMZfIq5TDmWGz7wsXUYw/Kx5wYUpbYgQt/ckkZQfvXQL0oZ+ASt58a4Zw==;EndpointSuffix=core.windows.net'
CONTAINER_NAME = 'incoming'
BLOB_NAME = '/student_habits_performance.csv'

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONN_STRING)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=BLOB_NAME)

# Download the CSV file to a local path
local_path = '/tmp/data'
os.makedirs(local_path, exist_ok=True)
download_file_path = os.path.join(local_path, 'student_habits_performance.csv')

with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

# Initialize Spark session
spark = SparkSession.builder.appName("AzureStorageToUnityCatalog").getOrCreate()

# Read the dataset into a Spark DataFrame
df = spark.read.csv(download_file_path, header=True, inferSchema=True)

# Display the DataFrame to ensure it is read correctly
display(df)

df.printSchema()

student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4
S1005,24,Male,7.2,1.3,0.0,No,82.9,7.4,Fair,1,Master,Average,4,No,100.0
S1006,21,Female,5.6,1.5,1.4,Yes,85.8,6.5,Good,2,Master,Poor,4,No,89.8
S1007,21,Female,4.3,1.0,2.0,Yes,77.7,4.6,Fair,0,Bachelor,Average,8,No,72.6
S1008,23,Female,4.4,2.2,1.7,No,100.0,7.1,Good,3,Bachelor,Good,1,No,78.9
S1009,18,Female,4.8,3.1,1.3,No,95.4,7.5,Good,5,Bachelor,Good,10,Yes,100.0


root
 |-- student_id: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- study_hours_per_day: double (nullable = true)
 |-- social_media_hours: double (nullable = true)
 |-- netflix_hours: double (nullable = true)
 |-- part_time_job: string (nullable = true)
 |-- attendance_percentage: double (nullable = true)
 |-- sleep_hours: double (nullable = true)
 |-- diet_quality: string (nullable = true)
 |-- exercise_frequency: integer (nullable = true)
 |-- parental_education_level: string (nullable = true)
 |-- internet_quality: string (nullable = true)
 |-- mental_health_rating: integer (nullable = true)
 |-- extracurricular_participation: string (nullable = true)
 |-- exam_score: double (nullable = true)



## Data Exploration

In [0]:
# Display the descriptive statistics for the 'parental_education_level' column
display(df.describe('parental_education_level'))

summary,parental_education_level
count,1000
mean,null
stddev,null
min,Bachelor
max,None


## Filling missing values

In [0]:
mode_value = df.groupBy('parental_education_level') \
               .count() \
               .orderBy('count', ascending=False) \
               .first()[0]

df = df.fillna({'parental_education_level': mode_value})
display(df)

student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4
S1005,24,Male,7.2,1.3,0.0,No,82.9,7.4,Fair,1,Master,Average,4,No,100.0
S1006,21,Female,5.6,1.5,1.4,Yes,85.8,6.5,Good,2,Master,Poor,4,No,89.8
S1007,21,Female,4.3,1.0,2.0,Yes,77.7,4.6,Fair,0,Bachelor,Average,8,No,72.6
S1008,23,Female,4.4,2.2,1.7,No,100.0,7.1,Good,3,Bachelor,Good,1,No,78.9
S1009,18,Female,4.8,3.1,1.3,No,95.4,7.5,Good,5,Bachelor,Good,10,Yes,100.0


## Counting null values

In [0]:
null_counts = df.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]
)
display(null_counts)

student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Writing cleaned data to Data Warehouse

In [0]:
# Define Unity Catalog table name
catalog_name = "novi_cleaned"
schema_name = "cleaned"
table_name = "habits_cleaned"

# Write the DataFrame to a Unity Catalog table
df.write.saveAsTable(f"{catalog_name}.{schema_name}.{table_name}", mode="overwrite")
21A
# Display success message
displayHTML("<p>Dataset uploaded to Data Warehouse table successfully!</p>")

Dataset uploaded to Data Warehouse table successfully!